### XOR with fixed length key

Testing out how the original code works

In [21]:
import os, random, hashlib
from itertools import cycle

pt   = [i%230 for i in os.urandom(500000)]
klen = random.randint(1000,10000)
key  = random.getrandbits(klen*8).to_bytes(klen, 'big')
print('klen', klen)
print('key', key.hex())

#open('ct', 'wb').write(bytes(c^i for c,i in zip(pt, cycle(key))))
ct = (bytes(c^i for c,i in zip(pt, cycle(key))))
print('ct', ct.hex())

flag = "greyhats{%s}"%hashlib.sha256(key).hexdigest()
print('flag', flag)

klen 9949
key 2605a77f57d3c1f23b30d23e5fd425e7a4af7bd55b64d81b0db15b2e534968518ae9e170268010c65e9e191535ab6d90537a0f0dc8445a69efbe4c48d2554df7e90f978943fe1434afd74e4806ef920c15d15b0e2d40a82e581481c82e05e9d951ca9bc5d77fc9898cf8395fc17a738eb38b2a04da5cca563eca01c3f0b944efe29658a9bcc417b5c1f9ad604bf3cfa04ec9199b4d7d39478955d0e9d033af3818cbddebbf39ad2f4b9def78c2d1263ca67c173526113e1851b5f09a37e52f32daba93b2a3689c7ed440518a031da0d54a0322e61c1a8c6f38ea8ff771f4f18d7738434d71e9f95fd0ac308171c8b899574fed97c24c2be5eda76dbf94c256c57faf9e69b78ef5b7b89de45666c1cb290952fd1f819f60aed50ca57243721e82f98d2e76491fa2ebff4fb5b856a985980e0c7001241d9dd4ab00c049bc51df05e70469603273eaf6769de266eac6ed143f4bd1d94b6ef8dc727ad90ed004d30d133a4e5204133c2a14d86d51b7e71db3cac3737c79e4f837e22cef719bf0419353e9b49728eea783bfc55673e0a5574af289c722ab540c00c902f731ab39cdaf5546bf1c92059edf2ac6cc2048b8b562c39c93b8b58463ad8f2d5f9d06328658407519b523ab983631820cc66bce245b673e3644657d7db2a62f92362e457d489eb1efb1e3d45ca03ff5c6ad84

Now lets put in what we have.

In [1]:
# Given ct
ct = open('xor_fixed_key_ct', 'rb').read()
print('ct', ct.hex())
print('ct_len', len(ct))

ct ...
ct_len 500000


Define a naive solving function 

In [2]:
# Guess a klen, then try to fit every byte knowing that it is under modulo 230.
def extract_key(ct, keylen, reverse_search=False):
    # At each offset,
    key_found = []
    for offset in range(keylen):
        print(f"\rTrying offset {offset}", end="")
        ci = list(ct[offset::keylen])
        # Find a key that allows pt modulo 230 to be fulfilled
        ki = None
        myrange = reversed(range(255)) if reverse_search else range(255)
        for k in myrange:
            pi = [c ^ k for c in ci]
            if max(pi) < 230:
                ki = k
                break
        # if cannot find at all, then fail the whole extraction
        if ki == None:
            return None
        # else add it to our progress and continue
        else:
            key_found.append(k)
    return key_found


Run a loop, so we can narrow down the key length

In [3]:
for kl in range(1000, 10000):
    mykey = extract_key(ct, kl)
    if mykey:
        mykey = bytes(mykey)
        mykey1 = mykey.hex()
        print(f"\r", end="")
        print('keylen', kl, 'mykey', mykey1)


keylen 7347 mykey 80380088c8c0c2a090d27208082ac06aa800a0e858d25060c8745820724840929800d02040b8c0a0f07820ea6a004a08b0489268bceae474a0de562af8d240c26acab2f6e8d000e8c0faa298eac0a2008aba32f8e2600ac09004c290ba5220009a30945ec03e28b8b0a88022406cc0803a7018b81a2448000afce092da2a40a0aa447a60788012e000dab8c2b04894e08804b260684280d02830faf0d810800cfe38a0a008182090b058c84af8944060c860bad80060922028006af2e0c210181220b072a0f848001038d8807850a01042e8d052987820d07a824018e2e8c28a5a58e8e2e040bc28d8e88ab8903c203280608078f8f0445858509ab2108878cc38fafe5018fce810e290a86080c260d06038627a1ae852b0e0c0006cc230608ec09880c0e0c06a7a409a80a8606258406290e03cd0ea32a008b850b282e0783a0ec008c0920040388082b238b86018e85a7aea48c02058605cc0a8cc22b630bca8ac60626062f05a2020b888d08298e01068c07820826888e2a010c8b8081ab8c00242603a44d8082af89812a02880106088a0e2e85c82ca345ae0b030f0c0c09858a098181058c058204028c0b860a018c070a0c00028004660e02858ea4060281a60d030b8ba00e81a80b040029088a0bcfcc0e078c032608098203818cc9cb020808adcc0fa4080a8cc

With this we know the key length is 7347 because it is the only one that passed

However realize that there are multiple possible keys. If we do a reverse search, we get a totally different key.

In [4]:
mykeyA = extract_key(ct, keylen=7347, reverse_search=False)
mykeyA = bytes(mykeyA)
print('A', mykeyA.hex())

mykeyB = extract_key(ct, keylen=7347, reverse_search=True)
mykeyB = bytes(mykeyB)
print('B', mykeyB.hex())


Trying offset 7346A 80380088c8c0c2a090d27208082ac06aa800a0e858d25060c8745820724840929800d02040b8c0a0f07820ea6a004a08b0489268bceae474a0de562af8d240c26acab2f6e8d000e8c0faa298eac0a2008aba32f8e2600ac09004c290ba5220009a30945ec03e28b8b0a88022406cc0803a7018b81a2448000afce092da2a40a0aa447a60788012e000dab8c2b04894e08804b260684280d02830faf0d810800cfe38a0a008182090b058c84af8944060c860bad80060922028006af2e0c210181220b072a0f848001038d8807850a01042e8d052987820d07a824018e2e8c28a5a58e8e2e040bc28d8e88ab8903c203280608078f8f0445858509ab2108878cc38fafe5018fce810e290a86080c260d06038627a1ae852b0e0c0006cc230608ec09880c0e0c06a7a409a80a8606258406290e03cd0ea32a008b850b282e0783a0ec008c0920040388082b238b86018e85a7aea48c02058605cc0a8cc22b630bca8ac60626062f05a2020b888d08298e01068c07820826888e2a010c8b8081ab8c00242603a44d8082af89812a02880106088a0e2e85c82ca345ae0b030f0c0c09858a098181058c058204028c0b860a018c070a0c00028004660e02858ea4060281a60d030b8ba00e81a80b040029088a0bcfcc0e078c032608098203818cc9cb020808adcc0fa4080a8

Too many permutations

In [39]:
# Count unmatched pairs

def bitstring(br):
    return ''.join(format((b), '08b') for b in br)

count = 0

offset = 624*55
bitsA = bitstring(mykeyA)[offset:offset+624*32]
bitsB = bitstring(mykeyB)[offset:offset+624*32]

for a, b in zip(bitsA, bitsB):
    if a != b:
        count += 1
print(f"Differs {count} out of {len(bitsB)}")


Differs 7222 out of 19968


---

To input these random states into randcrack, and solve for the most probable output

In [14]:
!pip install randcrack

Defaulting to user installation because normal site-packages is not writeable
